# Introduction <a name="introduction"></a>

This notebook outlines the procedures for managing inventories across various platforms. It includes detailed instructions for creating, updating, and managing inventories, as well as handling collaborator roles.

- **Inventory Management:** Enable the creation and management of inventory items while supporting multiple users with varied roles for seamless collaboration in inventory management.

**Required Services:**
- `watsonx.governance`

**Required Packages:**
- **IBM Facts Client Python SDK (>=1.0.80)**


In [ ]:
!pip install ibm-aigov-facts-client


In [ ]:
import shutil
from dotenv import load_dotenv
import os
from IPython.core.display import display, Markdown
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig

# warnings.filterwarnings('ignore')
shutil.rmtree('./mlruns', ignore_errors=True)
load_dotenv()

- This sample will use IBM Cloud by default. If you prefer to IBM watsonx.goverance software, set `use_software=True`

In [2]:
use_software=True

---
## Authentication Setup<a name="setup"></a>

### IBM Cloud  <a name="IBM-Cloud"></a>

Your Cloud API key can be generated by going to the Users section of the Cloud console. From that page, go to **Manage->Access(IAM)->API keys-> Create**. Give your key a name and click Create, then copy the created key and use as API_KEY.

NOTE: You can also get OpenScale API_KEY using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: instruction

How to get api key using console:

```
bx login --sso
bx iam api-key-create 'my_key'
```
- Get relevant space id from UI `(Deployments -> Spaces-> open space -> Manage -Space GUID)`

In [ ]:
if not use_software:
    API_KEY=os.getenv("CLOUD_API_KEY","<if you can't use .env you can provide your value here>")

[back to top](#introduction)
### IBM watsonx.goverance software<a name="Watsonx.Gov-Platform"></a>

- Service url is the watsonx.goverance software platform host URL. For skytap environment, it would be the internal nginx URL.
- You can either use user `password` or platform `apikey` to authenticate

In [ ]:
if use_software:
    creds=CloudPakforDataConfig(service_url=os.getenv("CPD_SERVICE_URL", "<if you can't use .env you can provide your value here>"),
        username=os.getenv("CPD_USERNAME", "<if you can't use .env you can provide your value here>"),
        password=os.getenv("CPD_PASSWORD", "<if you can't use .env you can provide your value here>"))


## Client Initialization





In [ ]:
if use_software:
   facts_client = AIGovFactsClient(cloud_pak_for_data_configs=creds,disable_tracing=True,external_model=True)
else: 
   facts_client = AIGovFactsClient(api_key=API_KEY,disable_tracing=True,external_model=True)

In [ ]:
facts_client.version 

---
## Inventory Management

This section focuses on the creation and management of inventory on the Watsonx.Governance platform.
The inventory is a view where you can define an AI use case to request a new model, and then track the model and related assets through its lifecycle

<h3 style="color:gold;">Create Inventory</h3>

This method is used to create an inventory item with a specified name and description.

If you are utilizing IBM Cloud, ensure that you provide the **`cloud_object_storage_name`** parameter to correctly associate the inventory item with a Cloud Object Storage (COS) instance.


#### -> Retrieves a list of cloud object storage instances.

In [ ]:
if not use_software:
    cloud_object_storage_details=facts_client.utilities.get_cloud_object_storage_instances()
    print(cloud_object_storage_details)


In [ ]:
if not use_software:
    def get_storage_name_by_name(cloud_details, name):
        for detail in cloud_details:
            if detail['Name'] == name:
                return detail['Name']  # Return only the name
        return None  # Return None if the name is not found
  
    
    # select the desired name from the above list 
    user_selected_name = 'CloudObjectStorage'
    cloud_object_storage_name = get_storage_name_by_name(cloud_object_storage_details, user_selected_name)

    if cloud_object_storage_name:
        print(cloud_object_storage_name)
    else:
        print(f"No cloud_object_storage found with the name '{user_selected_name}'")


In [ ]:
if not use_software:
    inventory=facts_client.assets.create_inventory(name="New Inventory",description="testing",cloud_object_storage_name=cloud_object_storage_name)
else:
    inventory=facts_client.assets.create_inventory(name="New Inventory",description="testing")

In [ ]:
inventory.get_info()

<h3 style="color:gold;">Retrieve a specific inventory by its Inventory ID. </h3>

This method fetches the details of a specific inventory item using its inventory_id.


#### -> List Inventories

In [ ]:
inventory_details=facts_client.assets.list_inventories()
print(inventory_details)



In [ ]:
desired_inventory_name = "New Inventory"

inventory_id = None
for inventory in inventory_details:
    if inventory.to_dict().get("inventory_name") == desired_inventory_name:
        inventory_id = inventory.to_dict().get("inventory_id")
        break 

print(inventory_id)

#### -> Get Inventory

In [ ]:
inventory=facts_client.assets.get_inventory(inventory_id=inventory_id)


In [ ]:
inventory.get_info()

<h3 style="color:gold;">Set Details of an Inventory Item </h3>

It allows updating the name and/or description of the inventory.

In [ ]:
inventory.set_detail(name="New Inventory Name",description="New Inventory Description")

In [ ]:
inventory.get_info()

---

<h2>Inventory Collaborators</h2>

Inventories are inherently collaborative, allowing multiple users with distinct roles to participate in the inventory.

This section provides a detailed overview of the methods available for managing collaborators within each inventory item, including:

- Viewing the current list of collaborators
- Assigning new collaborators
- Defining roles for each collaborator
- Removing existing collaborators




<h3 style="color:gold;">Retrieve a list of collaborators for the inventory. </h3>

In [ ]:
# Get the list of collaborators
inventory.list_collaborators()

---

<h3 style="color:gold;">Assigning New Collaborators</h3>

To add new collaborators, you need either the `user_id` or the `access_group_id`, depending on the platform you're using.

#### In the Watsonx.Gov Platform:
You need either the `user_id` or the `access_group_id`. To find these:

1. **Find the `user_id`:**
   - Go to the Watsonx.Gov Platform.
   - Navigate to **Access Control**.
   - Select **Users**.
   - Locate the `User-ID` for the desired user.

2. **Find the `access_group_id`:**
   - Go to the Watsonx.Gov Platform.
   - Navigate to **Access Control**.
   - Select **Users Group**.
   - Find the desired group.
   - The `access_group_id` is located at the end of the URL, e.g., `usermgmt-ui/groups/10001`, where `10001` is the group ID.

#### In the  IBM Cloud:
You need either the `user_iam_id` or the `access_group_id`. To find these:

1. **Find the `user_iam_id`:**
   - Go to [cloud.ibm.com](https://cloud.ibm.com).
   - Navigate to **IAM**.
   - Select **Users**.
   - Click on the three dots next to the user and choose **Manage User**. This will allow you to retrieve the necessary `user_iam_id` to add collaborators.

2. **Find the `access_group_id`:**
   - Go to [cloud.ibm.com](https://cloud.ibm.com).
   - Navigate to **IAM**.
   - Select **Access Groups**.
   - Click on the three dots next to the group and choose **Manage Access**. 
   - This will allow you to retrieve the necessary `access_group_id` from the details to add collaborators.




In [ ]:
# Determine user_id and access_group_id based on use_software flag
if use_software:
    user_id = "1000000**"
    access_group_id = "1000*"
else:
    user_id = "IBMid-69*******"
    access_group_id = "AccessGroupId-**********"


In [ ]:
if user_id:
    inventory.add_collaborator(user_id=user_id, role="editor")
elif access_group_id:
    inventory.add_collaborator(access_group_id=access_group_id, role="editor")
else:
    raise ValueError("Either user_id or access_group_id must be provided")

<h3 style="color:gold;">Settting the role of an existing collaborator in the inventory</h3>

This method changes the role assigned to a user in the inventory

In [ ]:
# desired collaborator name 
collaborator_name="Software Developer"

In [ ]:
# Get the list of collaborators
collaborators = inventory.list_collaborators()
collaborator_id = None
access_group_id = None

for collaborator in collaborators:
    name_or_id = collaborator.get("name")
    
    if name_or_id == collaborator_name:
        collaborator_id = collaborator.get("user_id")
        if not collaborator_id:
            access_group_id = collaborator.get("access_group_id")
        break

# Print the appropriate ID if found
if collaborator_id:
    print(f"User ID: {collaborator_id}")
elif access_group_id:
    print(f"Access Group ID: {access_group_id}")
else:
    print(f"Collaborator with name or access group ID {collaborator_name=} not found.")


In [ ]:
inventory.set_collaborator_role(user_id=collaborator_id,access_group_id=access_group_id,role="admin")

<h3 style="color:gold;">Deleting existing collaborators</h3>

This method removes a user from the inventory.

In [ ]:
inventory.remove_collaborator(user_id=collaborator_id,access_group_id=access_group_id)

---
## Delete Inventory

This section covers the process for deleting an inventory item. Make sure to carefully review the item before proceeding, as deletion is irreversible.


In [ ]:
inventory.delete_inventory()

**Created by:**  


IBM watsonx.governance - AI Factsheet Python SDK Team

---

**Copyright © 2020-2024 IBM**  
Released under the MIT License.
